In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
# Function to initialize WebDriver
def init_driver():
    driver = webdriver.Chrome()  # Or replace with your WebDriver path
    driver.maximize_window()
    return driver

In [ ]:
# Function to scroll to an element
def scroll_to_element(driver, element):
    """Scroll the page to make the element visible."""
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()

In [ ]:
# Function to navigate through pages and scrape bus details
def navigate_through_pages(driver, base_url):
    driver.get(base_url)  # Visit the base URL
    """Scrape bus details from the TC page."""
    driver.get(base_url)
    time.sleep(5)  # Allow page to load initially

    while True:
        try:
            # Wait for the bus list to load
            try:
                WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route_link")))
            except Exception as e:
                print("Bus details did not load properly:", e)
                return []

            # Scrape bus details
            all_data = []
            buses = driver.find_elements(By.CLASS_NAME, "route_link")
            print(len(buses))
                
            # Wait for the pagination buttons to load (using a unique identifier like class name or XPath)
    
            page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
            
            # Find the active page (the one with class 'DC_117_pageActive')
            active_page = None
            for page_tab in page_tabs:
                if "DC_117_pageActive" in page_tab.get_attribute("class"):
                    active_page = page_tab
                    break

            if not active_page:
                print("No active page found, unable to navigate.")
                break

            # Print active page number
            active_page_number = active_page.text
            print(f"Currently on page: {active_page_number}")
            
            # Try to locate the "next" page button/tab (it could be the next sibling of the active page tab)
            next_page = None
            active_index = page_tabs.index(active_page)
            
            # Check if next page exists
            if active_index < len(page_tabs) - 1:
                next_page = page_tabs[active_index + 1]

            if next_page:
                # Scroll to the next page and click
                print(f"Scrolling to next page: {next_page.text}")
                scroll_to_element(driver, next_page)  # Scroll to the "next" button
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(next_page))
                next_page.click()
                
                # Wait for page to refresh (or for the stale element to disappear)
                """WebDriverWait(driver, 10).until(EC.staleness_of(active_page))"""
                time.sleep(3)  # Give time for new content to load
            else:
                print("No more pages to navigate.")
                break  # Exit loop if there is no next page

        except NoSuchElementException:
            print("Pagination elements not found.")
            break
        except StaleElementReferenceException:
            print("Stale element found, retrying...")
            continue
        except ElementClickInterceptedException as e:
            print(f"Click interception error: {e}")
            continue


In [ ]:
# Main execution function
def main():
    base_url = "https://www.redbus.in/online-booking/gsrtc"  # URL of the page to scrape
    driver = init_driver()

    try:
        # Navigate through pagination and scrape data
        navigate_through_pages(driver, base_url)
    except Exception as e:
        print(f"Error during scraping: {e}")
    finally:
        driver.quit()  # Ensure that the driver quits after execution

In [ ]:
# Run the main function
if __name__ == "__main__":
    main()